In [ ]:
!pip install xmltodict

In [1]:
import xml.etree.ElementTree as ElementTree
import xmltodict
import pprint
import json
import pandas as pd
import numpy as np

class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)


class XmlDictConfig(dict):
    def __init__(self, parent_element):
        if parent_element.items():
            self.updateShim( dict(parent_element.items()) )
        for element in parent_element:
            if len(element):
                aDict = XmlDictConfig(element)
            #   if element.items():
            #   aDict.updateShim(dict(element.items()))
                self.updateShim({element.tag: aDict})
            elif element.items():    # items() is specialy for attribtes
                elementattrib= element.items()
                if element.text:           
                    elementattrib.append((element.tag,element.text ))     # add tag:text if there exist
                self.updateShim({element.tag: dict(elementattrib)})
            else:
                self.updateShim({element.tag: element.text})

    def updateShim (self, aDict ):
        for key in aDict.keys():   # keys() includes tag and attributes
            if key in self:
                value = self.pop(key)
                if type(value) is not list:
                    listOfDicts = []
                    listOfDicts.append(value)
                    listOfDicts.append(aDict[key])
                    self.update({key: listOfDicts})
                else:
                    value.append(aDict[key])
                    self.update({key: value})
            else:
                self.update({key:aDict[key]})



# Open an output file

In [2]:
tree = ElementTree.parse('./raw/BUILT.30.experienced_plans.xml')
root = tree.getroot()
xmldict = XmlDictConfig(root)


In [3]:
#initiate activity related list object 
id_data_act = []
activity_data = []
start_time = []
end_time = []
for type_tag in root.findall('person'):
    for act in type_tag.findall('plan/activity'):
        id_data_act.append(type_tag.get('id')) 
        activity_data.append(act.get('type'))
        start_time.append(act.get('start_time'))
        end_time.append(act.get('end_time'))

#initiate mode related list object 
id_data_mode = []
mode_data = []

for type_tag in root.findall('person'):
    for i ,mode in enumerate(type_tag.findall('plan/leg')):
        id_data_mode.append(type_tag.get('id')) 
        mode_data.append(mode.get('mode'))

In [4]:
actList =  list(zip(id_data_act, activity_data, start_time, end_time))
modeList = list(zip(id_data_mode, mode_data))

df_act = pd.DataFrame(actList, columns=['id','activity','start_time','end_time'])
df_mode = pd.DataFrame(modeList, columns=['id','mode'])

#left join 
merge = pd.merge(df_act, df_mode, how='left', left_on='id', right_on='id')
df_unique = merge.drop_duplicates(subset=['id','activity','start_time','end_time'],keep='first')

In [5]:
# change the type of time
df_unique['start_time']=pd.to_datetime(df_unique['start_time'])
df_unique['end_time']=pd.to_datetime(df_unique['end_time'])


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Calculate the trave time for each trip

In [7]:
# Calculate the travel time
trave_time = pd.DataFrame(columns={'id','travel_time'})

for i in range(0,len(df_unique)-1):
    if(df_unique.iloc[i]['id']==df_unique.iloc[i+1]['id']):
        diff = pd.Timedelta(df_unique.iloc[i]['end_time']  - df_unique.iloc[i+1]['start_time'] ).seconds / 3600
    else:
        diff = np.nan
    
    trave_time=trave_time.append([{'id': df_unique.iloc[i]['id'], 'travel_time':diff}], ignore_index=True)

#merge 
df_final =  pd.concat([df_unique, trave_time['travel_time']], axis=1)
df_final


,id,activity,start_time,end_time,mode,travel_time
0,0,Work,NaT,2019-04-30 09:03:00,bike,22.695556
1,NaN,NaN,NaT,NaT,NaN,23.913611
2,NaN,NaN,NaT,NaT,NaN,23.837222
3,NaN,NaN,NaT,NaT,NaN,22.654722
4,0,Work,2019-04-30 10:21:16,2019-04-30 13:31:16,bike,NaN
5,NaN,NaN,NaT,NaT,NaN,23.643611
6,NaN,NaN,NaT,NaT,NaN,NaN
7,NaN,NaN,NaT,NaT,NaN,23.622500
8,0,Secondary,2019-04-30 13:36:27,2019-04-30 13:42:27,bike,19.706944
9,NaN,NaN,NaT,NaT,NaN,NaN


## Calculate the Total travel time of each ID

In [19]:
# Group by
total_travel_time = df_final.groupby(['id']).sum()
total_travel_time


NameError: name 'df_final' is not defined

## Export Excel File

In [20]:
df_final.to_excel("10th_mode_final.xlsx")
total_travel_time.to_excel("10th_totalTime_final.xlsx")
print("export succeed")


NameError: name 'df_final' is not defined